In [ ]:
!pip install transformers diffusers accelerate peft

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from huggingface_hub import hf_hub_download

realistic_vision_path = hf_hub_download(repo_id="SG161222/Realistic_Vision_V5.1_noVAE", filename="Realistic_Vision_V5.1-inpainting.safetensors")
vae_path = hf_hub_download(repo_id="stabilityai/sd-vae-ft-mse-original", filename="vae-ft-mse-840000-ema-pruned.safetensors")

In [3]:
import sys
sys.path.append('../src')

In [4]:
from config import DatasetConfig, Config, ModelConfig, WandbConfig, EvaluationConfig, TrainConfig

dataset_config = DatasetConfig(
    roboflow_api_key='HNXIsW3WwnidNDQZHexX',
    roboflow_workspace='arked',
    project_name='facades-flzke',
    dataset_version=11,
    data_root='facades_data',
    image_size=512,
)

model_config = ModelConfig(
    model_path=realistic_vision_path,
    vae_path=vae_path,
)

wandb_config = WandbConfig(
    project_name='facades',
)

eval_config=EvaluationConfig(
    prompts=['white facade', 'brick facade'],
)

train_config=TrainConfig(
    checkpoint_folder = wandb_config.project_name + "_checkpoints",
    train_batch_size = 4,
    unet_lr=1e-4,
    text_encoder_lr=1e-4,
    scheduler_num_cycles=4,
    total_steps=1000,
)

config = Config(
    dataset=dataset_config,
    model=model_config,
    wandb=wandb_config,
    eval=eval_config,
    train=train_config,
)

In [5]:
from model import get_models

text_encoder, vae, unet, tokenizer, noise_scheduler, placeholder_token_ids = get_models(
    config.model.model_path,
    config.model.vae_path,
    device=config.device,
    load_from_safetensor=True,
)

loading VAE...
loading model...


`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
c:\Users\Facundo\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\models\clip\feature_extraction_clip.py:28: FutureWarning: The class CLIPFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use CLIPImageProcessor instead.
  warnings.warn(


In [6]:
from peft import LoraConfig, LoraModel

unet_peft = LoraConfig(
    r=config.lora.rank,
    lora_alpha=config.lora.alpha,
    target_modules=config.lora.unet_target_modules,
    lora_dropout=0.1,
    bias='none',
)

text_encoder_peft = LoraConfig(
    r=config.lora.rank,
    lora_alpha=config.lora.alpha,
    target_modules=config.lora.text_encoder_target_modules,
    lora_dropout=0.1,
    bias='none',
)


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
For effortless bug reporting copy-paste your error into this form: https://docs.google.com/forms/d/e/1FAIpQLScPB8emS3Thkp66nvqwmjTEgxp8Y9ufuWTzFyr9kJ5AoI47dQ/viewform?usp=sf_link
CUDA SETUP: Loading binary c:\Users\Facundo\AppData\Local\Programs\Python\Python310\lib\site-packages\bitsandbytes\libbitsandbytes_cuda116.dll...


In [7]:
config.lora.text_encoder_adapter_name

'lora_te'

In [8]:
text_encoder = LoraModel(text_encoder, text_encoder_peft, config.lora.text_encoder_adapter_name)
unet = LoraModel(unet, unet_peft, config.lora.unet_adapter_name)

In [9]:
from utils import print_trainable_parameters

print_trainable_parameters(text_encoder, 'Text Encoder')
print_trainable_parameters(unet, 'UNet')

Model: Text Encoder
trainable params: 1327104 || all params: 124387584 || trainable%: 1.0669103437204794
Model: UNet
trainable params: 2492928 || all params: 862028292 || trainable%: 0.28919329250970804


In [10]:
import itertools

params_to_optimize = [
    {
        "params": itertools.chain(unet.parameters()),
        "lr": config.train.unet_lr,
    },
]

params_to_optimize += [
    {
        "params": itertools.chain(text_encoder.parameters()),
        "lr": config.train.text_encoder_lr,
    }
]

In [11]:
from torch.optim import AdamW

optimizer_lora = AdamW(
    params_to_optimize,
    lr=config.train.learning_rate,
    weight_decay=config.train.weight_decay,
)

In [12]:
text_encoder.peft_config

{'lora_te': LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type=None, inference_mode=False, r=8, target_modules={'mlp.fc1', 'mlp.fc2', 'k_proj', 'v_proj', 'out_proj', 'q_proj'}, lora_alpha=32.0, lora_dropout=0.1, fan_in_fan_out=False, bias='none', modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={})}

In [21]:
from utils import save_loras

save_loras(
    unet=unet, 
    text_encoder=text_encoder, 
    save_path='te.safetensors',
    config=config
)

In [22]:
from safetensors import safe_open

file = safe_open('te.safetensors', framework="pt", device=0)
file.keys()

['lora_te_text_model_encoder_layers_0_mlp_fc1.alpha',
 'lora_te_text_model_encoder_layers_0_mlp_fc1.lora_down.weight',
 'lora_te_text_model_encoder_layers_0_mlp_fc1.lora_up.weight',
 'lora_te_text_model_encoder_layers_0_mlp_fc2.alpha',
 'lora_te_text_model_encoder_layers_0_mlp_fc2.lora_down.weight',
 'lora_te_text_model_encoder_layers_0_mlp_fc2.lora_up.weight',
 'lora_te_text_model_encoder_layers_0_self_attn_k_proj.alpha',
 'lora_te_text_model_encoder_layers_0_self_attn_k_proj.lora_down.weight',
 'lora_te_text_model_encoder_layers_0_self_attn_k_proj.lora_up.weight',
 'lora_te_text_model_encoder_layers_0_self_attn_out_proj.alpha',
 'lora_te_text_model_encoder_layers_0_self_attn_out_proj.lora_down.weight',
 'lora_te_text_model_encoder_layers_0_self_attn_out_proj.lora_up.weight',
 'lora_te_text_model_encoder_layers_0_self_attn_q_proj.alpha',
 'lora_te_text_model_encoder_layers_0_self_attn_q_proj.lora_down.weight',
 'lora_te_text_model_encoder_layers_0_self_attn_q_proj.lora_up.weight',
 'l

In [19]:
from safetensors import safe_open

file = safe_open('test_lora.safetensors', framework="pt", device=0)

In [20]:
file.keys()

['lora_te_text_model_encoder_layers_0_self_attn_k_proj.alpha',
 'lora_te_text_model_encoder_layers_0_self_attn_k_proj.lora_down.weight',
 'lora_te_text_model_encoder_layers_0_self_attn_k_proj.lora_up.weight',
 'lora_te_text_model_encoder_layers_0_self_attn_out_proj.alpha',
 'lora_te_text_model_encoder_layers_0_self_attn_out_proj.lora_down.weight',
 'lora_te_text_model_encoder_layers_0_self_attn_out_proj.lora_up.weight',
 'lora_te_text_model_encoder_layers_0_self_attn_q_proj.alpha',
 'lora_te_text_model_encoder_layers_0_self_attn_q_proj.lora_down.weight',
 'lora_te_text_model_encoder_layers_0_self_attn_q_proj.lora_up.weight',
 'lora_te_text_model_encoder_layers_0_self_attn_v_proj.alpha',
 'lora_te_text_model_encoder_layers_0_self_attn_v_proj.lora_down.weight',
 'lora_te_text_model_encoder_layers_0_self_attn_v_proj.lora_up.weight',
 'lora_te_text_model_encoder_layers_10_self_attn_k_proj.alpha',
 'lora_te_text_model_encoder_layers_10_self_attn_k_proj.lora_down.weight',
 'lora_te_text_mode

In [ ]:
text_encoder_lora.peft_config['text_encoder_lora'].lora_alpha

In [ ]:
file.get_tensor('lora_te_text_model_encoder_layers_0_self_attn_k_proj.alpha')